In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd



In this project, we will use the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake. For a reference, check the paper below:

- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.


In [16]:
# Run this cell to import the Eurosat data

def load_eurosat_data():
    data_dir = 'data/'
    x_train = np.load(os.path.join(data_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
    x_test  = np.load(os.path.join(data_dir, 'x_test.npy'))
    y_test  = np.load(os.path.join(data_dir, 'y_test.npy'))
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [17]:

def get_new_model(input_shape):
    """
    This function builds a Sequential model. A CNN with 6 layers. The function also compiles the model.
    """
    model = Sequential([
        Conv2D(filters = 16, kernel_size = (3,3), input_shape = input_shape, 
               activation = 'relu', padding = 'SAME', name = 'conv_1'),
        Conv2D(filters = 8, kernel_size = (3,3), padding = 'SAME', activation = 'relu', name = 'conv_2'),
        MaxPooling2D(pool_size = (8,8), name = 'pool_1'),
        Flatten(name = 'flatten'),
        Dense(units = 32, activation = 'relu', name = 'dense_1'),
        Dense(units = 10, activation = 'softmax', name = 'dense_2')
    ])
    
    
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model
    
# create the model

model = get_new_model(x_train[0].shape)
print("model input shape is ", model.input_shape[1:])
print("data sample training shape: ", x_train[0].shape)

In [19]:
# a function to evaluate a model's test accuracy

def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

# Compute the model's initial test accuracy

model.summary()
get_test_accuracy(model, x_test, y_test)

In [30]:

def get_checkpoint_every_epoch():
    """
    This function returns a ModelCheckpoint object, saving only weights at every epoch separately.
    """
    checkpoint_path = 'checkpoints_every_epoch/checkpoint_{epoch: 03d}'
    checkpoint_every_epoch = ModelCheckpoint(filepath = checkpoint_path,
                                            frequency = 'epoch', save_weights_only = True, verbose = 1)
    
    return  checkpoint_every_epoch



def get_checkpoint_best_only():
    """
    This function returns a ModelCheckpoint object, saving only weights if validation accuracy is the best. 
    """
    
    checkpoint_best_path = 'checkpoints_best_only/checkpoint'
    
    checkpoint_best_only = ModelCheckpoint(filepath = checkpoint_best_path,
                                          save_weights_only = True,
                                          monitor = 'val_accuracy', save_best_only = True, verbose = 1)
    
    
    return checkpoint_best_only 



def get_early_stopping():
    """
    This function returns an EarlyStopping object, monitoring validation accuracy to track 
    improvements over the last 3 epochs. 
    """
    
    early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 3)
    
    return early_stopping
      
    

In [23]:
# Create the callbacks

checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

In [12]:
# Train the model

callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
3968/4000 [============================>.] - ETA: 0s - loss: 1.9697 - accuracy: 0.2349
Epoch 00001: saving model to checkpoints_every_epoch/ checkpoint_ 01

Epoch 00001: val_accuracy improved from -inf to 0.33600, saving model to checkpoints_best_only/checkpoint
4000/4000 [==============================] - 83s 21ms/sample - loss: 1.9653 - accuracy: 0.2360 - val_loss: 1.6854 - val_accuracy: 0.3360
Epoch 2/50
3968/4000 [============================>.] - ETA: 0s - loss: 1.5779 - accuracy: 0.3947
Epoch 00002: saving model to checkpoints_every_epoch/ checkpoint_ 02

Epoch 00002: val_accuracy improved from 0.33600 to 0.39800, saving model to checkpoints_best_only/checkpoint
4000/4000 [==============================] - 80s 20ms/sample - loss: 1.5773 - accuracy: 0.3943 - val_loss: 1.5099 - val_accuracy: 0.3980
Epoch 3/50
3968/4000 [============================>.] - ETA: 0s - loss: 1.4238 - accuracy: 0.4665
Epoch 00003: saving model to 

In [32]:


def get_model_last_epoch(model):
    """
    This function create a new instance of the CNN,
    load on the weights from the last training epoch, and return the model.
    """
    input_shape = model.input_shape[1:]
    new_model = get_new_model(input_shape)
    
    checkpoint_path = 'checkpoints_every_epoch/checkpoint_{epoch: 03d}'
    checkpoint_dir  = os.path.dirname(checkpoint_path)
    #last_checkpoint = tf.train.last_checkpoint('checkpoints_every_epoch')
    latest = tf.train.latest_checkpoint(checkpoint_dir) 
    new_model.load_weights(latest)
    
    return new_model
    
    
    
def get_model_best_epoch(model):
    """
    This function creates a new instance of the CNN, load 
    on the weights leading to the highest validation accuracy, and return the model.
    """
    
    input_shape = model.input_shape[1:]
    new_model = get_new_model(input_shape)
    
    checkpoint_best_path = 'checkpoints_best_only/checkpoint'
    
    new_model.load_weights(checkpoint_best_path)
    
    return new_model
    
    

In [33]:
# Load the model weights and verify that the second has a higher validation accuarcy.

model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.716

Model with best epoch weights:
accuracy: 0.724


In [34]:

def get_model_eurosatnet():
    """
    This function returns the pretrained EuroSatNet.h5 model.
    """
    new_model = load_model('models/EuroSatNet.h5')
    
    return new_model
    
    

In [35]:
# Print a summary of the EuroSatNet model, along with its validation accuracy.

model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 16)        6416      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 32, 32, 16)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 16)        2320      
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 16)        6416      
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 16, 16, 16)        0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 16, 16, 16)      